## Creating sub-agent 3: Market analysis agent

In this folder we will create the second sub-agent for get options chains, insider trading transactions and get news. We will use the `financial dataset API` and three functions: `get_options_chains`, `get_insider_trading` and `get_news` using the `marketing-analyst-agent`.

### Prerequisites
---

You are going to install boto3 dependencies from pip. Make sure you have the latest version of it for full capabilities

Before running this notebook, make sure to populate the `.env` file with your `FINANCIAL_DATASETS_API_KEY` and `TAVILY_API_KEY`. These two `API` keys will be used by the agent to access information on user related questions. You can create an `API` here: https://www.financialdatasets.ai/

In [1]:
!pip uninstall boto3 botocore awscli --yes

Found existing installation: boto3 1.35.83
Uninstalling boto3-1.35.83:
  Successfully uninstalled boto3-1.35.83
Found existing installation: botocore 1.35.83
Uninstalling botocore-1.35.83:
  Successfully uninstalled botocore-1.35.83
Found existing installation: awscli 1.36.24
Uninstalling awscli-1.36.24:
  Successfully uninstalled awscli-1.36.24


In [2]:
# Install latest boto3
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt

### Import required libraries

Next we will import the required libraries. We will also import some support functions available in the parent directory. Those functions are:

1. `create_agent`: helps you to create the necessary IAM permissions and Bedrock agetns based on the agent's name, instructions, foundation models, descriptions and other properties.

1. `invoke_agent_helper`: helps you to invoke your agent using invoke_agent
You can see the implementation of both functions in the parent directory

In [3]:
import os
import sys
import time
import json
import boto3
import shutil
import logging
import zipfile
import subprocess
from dotenv import load_dotenv

# Get the current file's directory
current_dir = os.path.dirname(os.path.abspath('__file__'))

# Get the parent directory
parent_dir = os.path.dirname(current_dir)
print(parent_dir)

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from utils import *
from globals import *
from agents import create_agent, invoke_agent_helper

/Users/madhurpt/Desktop/hedge-fund-analyst-multi-agent-collaboration-2
Boto3 version: 1.35.83


[2024-12-17 19:44:08,151] p88722 {requests.py:227} INFO - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
[2024-12-17 19:44:08,278] p88722 {requests.py:263} INFO - <<< {"data":{"viewer":{"username":"madhur-prash","defaultEntity":{"name":"madhur-prash-none"}}}}
[2024-12-17 19:44:09,711] p88722 {utils.py:160} INFO - NumExpr defaulting to 8 threads.
[2024-12-17 19:44:10,521] p88722 {_client.py:1026} INFO - HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"
[2024-12-17 19:44:11,112] p88722 {config.py:59} INFO - PyTorch version 2.3.0 available.
[2024-12-17 19:44:11,336] p88722 {requests.py:227} INFO - >>> {"query": "query DefaultEntity {\n  viewer {\n    username\n    defaultEntity {\n      name\n    }\n  }\n}"}
[2024-12-17 19:44:11,428] p88722 {requests.py:263} INFO - <<< {"data":{"viewer":{"username":"madhur-prash","defaultEntity":{"name":"madhur-pr

Logged in as Weights & Biases user: madhur-prash.
View Weave data at https://wandb.ai/madhur-prash-none/multi-agent-collaboration/weave


In [4]:
# Load the environment variables that are defined in the ".env" file. This contains the 
# financial data API key that will enable the user to access the data
load_dotenv

<function dotenv.main.load_dotenv(dotenv_path: Union[str, ForwardRef('os.PathLike[str]'), NoneType] = None, stream: Optional[IO[str]] = None, verbose: bool = False, override: bool = False, interpolate: bool = True, encoding: Optional[str] = 'utf-8') -> bool>

In [5]:
# set a logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

### Defining agent configuration
---

In [6]:
agent_instruction = """You are a comprehensive market analysis assistant that helps users analyze options chains, insider trading data, and relevant market news.
You require the user to provide a stock ticker symbol for analysis.
If a user does not provide a ticker symbol, mention in the answer that they need to provide a ticker symbol.

You can perform the following types of analysis. You have access to APIs and tools to call to fetch data based on what the user question is:

1. Use the appropriate functions based on the analysis needed:
    - get_options_chain for options analysis
    - get_insider_trades for insider trading analysis
    - get_news for market news and sentiment

2. Options Chain Analysis:
   - View available options contracts
   - Filter by strike price
   - Filter by option type (call/put)
   - Analyze options pricing and volume

3. Insider Trading Analysis:
   - Recent insider transactions
   - Transaction types (buy/sell)

4. Market News Analysis:
   - Latest relevant news
   - Market sentiment
   - Industry trends

For options chain data, you can specify:
- Strike price filters
- Option type (call/put)
- Number of results to return (limit)

For insider trades, you can specify:
- Number of transactions to analyze (limit)

If you do not have access to the data that the user is asking for, do not make up an answer, just say that you do not know the answer. Be completely
accurate. Do not provide answers to anything but on the topic specified above.
"""

agent_description = "Agent for analyzing options chains, insider trading data, and market news"

In [7]:
# These are the functions that are used by the market analysis agent
functions = [{
    'name': 'get_options_chain',
    'description': 'Get options chain data for a ticker',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "limit": {
            "description": "number of options to retrieve",
            "required": False,
            "type": "integer"
        },
        "strike_price": {
            "description": "filter by strike price",
            "required": False,
            "type": "number"
        },
        "option_type": {
            "description": "filter by option type (call/put)",
            "required": False,
            "type": "string"
        }
    }
},
{
    'name': 'get_insider_trades',
    'description': 'Get insider trading transactions for a ticker',
    'parameters': {
        "ticker": {
            "description": "stock ticker symbol of the company",
            "required": True,
            "type": "string"
        },
        "limit": {
            "description": "number of transactions to retrieve",
            "required": False,
            "type": "integer"
        }
    }
},
{
    'name': 'get_news',
    'description': 'Get latest market news and analysis',
    'parameters': {
        "query": {
            "description": "search query for news",
            "required": True,
            "type": "string"
        },
        "max_results": {
            "description": "maximum number of news results",
            "required": False,
            "type": "integer"
        }
    }
}]

### Add a lambda layer to the lambda function
---

In this section of the notebook, we will add a lambda layer to the lambda function. We will be installing `requests` to make requests to the financial data API key.

In [8]:
# Create and publish the layer 
layer_zip = create_lambda_layer(['requests', 'tavily-python'])
layer_arn = publish_layer('marketing-agent-lambda-layer')

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
langchain-community 0.2.17 requires langchain-core<0.3.0,>=0.2.39, but you have langchain-core 0.3.24 which is incompatible.
langchain-community 0.2.17 requires langsmith<0.2.0,>=0.1.112, but you have langsmith 0.2.3 which is incompatible.
langchain-community 0.2.17 requires tenacity!=8.4.0,<9.0.0,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.28.3 which is incompatible.
streamlit 1.30.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
cohere 5.3.4 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.20.1 which is incompatible.
llam

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 982.4/982.4 kB 15.3 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
langchain-community 0.2.17 requires langchain-core<0.3.0,>=0.2.39, but you have langchain-core 0.3.24 which is incompatible.
langchain-community 0.2.17 requires langsmith<0.2.0,>=0.1.112, but you have langsmith 0.2.3 which is incompatible.
langchain-community 0.2.17 requires tenacity!=8.4.0,<9.0.0,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
streamlit 1.30.0 requires packaging<24,>=16.8, but you have packaging 24.2 which is incompatible.
streamlit 1.30.0 requires protobuf<5,>=3.20, but you have protobuf 5.28.3 which is incompatible.
streamlit 1.30.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
cohere 5.3.4 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.20.1 which is incompatible.
lang

### Create agent
---

Next we will create the agent with the provided information

In [9]:
action_group_config = {
    'name': 'MarketingAnalysisActionGroup',
    'description': 'Action group to analyze marketing questions from the user',
    'functions': functions,
    'lambda_function_name': f'{SUB_AGENT_MARKETING_ANALYST}-lambda',
    'lambda_file_path': MARKET_ANALYSIS_LAMBDA_FUNCTION_NAME,
    'dynamodb_table_name': f'{SUB_AGENT_MARKETING_ANALYST}-table',
    'dynamodb_attribute_name': 'analysis-id', 
    # Adding the lambda layer that contains the requests installation
    'lambda_layers': [layer_arn]
}

# Create a Lambda client and attach the API key as env variable to the lambda function
lambda_client = boto3.client('lambda')
lambda_function_name = action_group_config['lambda_function_name']
environment_variables = {
    'FINANCIAL_DATASETS_API_KEY': os.getenv('FINANCIAL_DATASETS_API_KEY'), 
    'TAVILY_API_KEY': os.getenv('TAVILY_API_KEY')
}

# Create the agent with the updated action group config
marketing_agent_id, marketing_agent_alias_id, marketing_agent_alias_arn = create_agent(
    SUB_AGENT_MARKETING_ANALYST,
    agent_instruction,
    agent_foundation_model=BEDROCK_MODEL_CLAUDE_HAIKU,
    agent_description=agent_description,
    action_group_config=action_group_config
)

creating agent
Policy market-analysis-agent-ba already exists
Checking if AmazonBedrockExecutionRoleForAgents_market-analysis-agent role also exists
Detaching and deleting market-analysis-agent-ba
deleting AmazonBedrockExecutionRoleForAgents_market-analysis-agent
Recreating market-analysis-agent-ba
creating and attaching action group
Role already exists -- deleting and creating it again
Detaching AWSLambdaBasicExecutionRole
Detaching AmazonDynamoDBFullAccess
deleting market-analysis-agent-lambda-role
recreating market-analysis-agent-lambda-role
attaching basic lambda permissions to market-analysis-agent-lambda-role
attaching dynamodb permissions to market-analysis-agent-lambda-role
market-analysis-agent-lambda already exists, deleting it and recreating
Agent id WBUINUADYX current status: NOT_PREPARED
Waiting for agent status to change. Current status PREPARING
Agent id WBUINUADYX current status: PREPARED
Waiting for agent status to change. Current status VERSIONING
Agent id WBUINUADYX 

In [10]:
# Update the Lambda function's configuration to include the environment variables
# In the case of this agent, we need the financial dataset and Tavily API as an env variable passed to the lambda
# so that it can be used in fetching data based on user questions
response = lambda_client.update_function_configuration(
    FunctionName=lambda_function_name,
    Environment={
        'Variables': environment_variables
    },
    Layers=action_group_config['lambda_layers']
)

### Getting details from the agent
--- 
Let's take a look at the details from the created agent.

In [11]:
marketing_agent_id, marketing_agent_alias_id, marketing_agent_alias_arn

('WBUINUADYX',
 '65H2UJP7TF',
 'arn:aws:bedrock:us-east-1:218208277580:agent-alias/WBUINUADYX/65H2UJP7TF')

### Testing the Fundamental Analyst agent
---

Now that we've created the agent, let's test it by using our invoke_agent_helper function. Here, we will invoke the `fundamental analyst` agent to provide balance sheet information, income statements and cash flow information based on the question provided by the user.

In [12]:
# Sleep for 30 seconds before invoking the marketing analyst agent
time.sleep(30)

### Route the agent to call the `get_news()` function
---

This function enables the agent with a `TAVILY_API_KEY` that enables web search. In our function, we have provided `google.com` and `bloomberg.com` as domains that the agent can access, so the agent will search for results in these domains and return a comprehensive answer. 

In [13]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """Can you give any news on 'AMZN' and what is its stock like? Search for 'Amazon stock'. Be concise and only search once."""
response = invoke_agent_helper(
    query, session_id, marketing_agent_id, marketing_agent_alias_id, enable_trace=True
)
print(response)

[2024-12-17 19:45:30,145] p88722 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:45:30,146] p88722 {agents.py:761} INFO - {
  "RequestId": "106ea156-49c8-494d-8055-02b340b3b9a6",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:45:30 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "106ea156-49c8-494d-8055-02b340b3b9a6",
    "x-amz-bedrock-agent-session-id": "616b2202-bcd9-11ef-a671-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:45:30,353] p88722 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:45:30,354] p88722 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/WBUINUADYX/65H2UJP7TF"
  }
]
[2024-12-17 19:45:30,354] p88722 {agents.py:803} INFO - Orchestration Trace Detected
[2024

🍩 https://wandb.ai/madhur-prash-none/multi-agent-collaboration/r/call/0193d73a-1c20-73a2-a1c5-d576b46eca73
Based on the news search, here's a summary of the key information on Amazon (AMZN) stock:

- Amazon's stock price and performance can be viewed on Google Finance at https://www.google.com/finance/quote/AMZN:NASDAQ. The latest news indicates that Amazon founder Jeff Bezos recently sold $3.4 billion worth of Amazon shares.

- The news also mentions that Amazon's video advertising business is expected to generate an extra $5 billion in revenue for the company.

- Overall, the news paints a positive picture of Amazon's stock performance and business outlook, though it does not provide a detailed analysis. The stock appears to be trading well and the company's various business segments, including advertising, are performing strongly.
CPU times: user 70.4 ms, sys: 22.7 ms, total: 93.2 ms
Wall time: 7.01 s


### Route the agent to call the `get_insider_trades()` function
---

In this question, we ask the agent about `How many shares of Apple has General Counsel sold over the past year?`. With this question, the agent identifies that `get_insider_trades()` is the right function to call and using the `FINANCIAL_DATASET_API` key, retrieves that information to answer the user question.

In [14]:
%%time
import uuid
session_id:str = str(uuid.uuid1())
query = """How many shares of AAPL has General Counsel sold over the past year?"""
response = invoke_agent_helper(
    query, session_id, marketing_agent_id, marketing_agent_alias_id, enable_trace=True
)
print(response)

[2024-12-17 19:45:37,137] p88722 {agents.py:760} INFO - invoke_agent response metadata:
[2024-12-17 19:45:37,138] p88722 {agents.py:761} INFO - {
  "RequestId": "b7018ab0-3d3c-4045-a5dc-17b25d26c2da",
  "HTTPStatusCode": 200,
  "HTTPHeaders": {
    "date": "Wed, 18 Dec 2024 00:45:37 GMT",
    "content-type": "application/vnd.amazon.eventstream",
    "transfer-encoding": "chunked",
    "connection": "keep-alive",
    "x-amzn-requestid": "b7018ab0-3d3c-4045-a5dc-17b25d26c2da",
    "x-amz-bedrock-agent-session-id": "659a5104-bcd9-11ef-a671-168aeb12c117",
    "x-amzn-bedrock-agent-content-type": "application/json"
  },
  "RetryAttempts": 0
}
[2024-12-17 19:45:37,300] p88722 {agents.py:787} INFO - ==== Trace Event Received ====
[2024-12-17 19:45:37,301] p88722 {agents.py:790} INFO - callerChain: [
  {
    "agentAliasArn": "arn:aws:bedrock:us-east-1:218208277580:agent-alias/WBUINUADYX/65H2UJP7TF"
  }
]
[2024-12-17 19:45:37,302] p88722 {agents.py:803} INFO - Orchestration Trace Detected
[2024

🍩 https://wandb.ai/madhur-prash-none/multi-agent-collaboration/r/call/0193d73a-3788-7371-ab1e-803037eab842
According to the insider trading data, Katherine L. Adams, the SVP, General Counsel and Secretary at Apple, has sold 8,000 shares of AAPL stock over the past year.
CPU times: user 59.6 ms, sys: 15.8 ms, total: 75.4 ms
Wall time: 2.95 s


### Saving the information
---
Let's now save some information so that we can use this agent as a sub-agent of our financial hedge fund assistant

In [15]:
%store marketing_agent_id
%store marketing_agent_alias_id
%store marketing_agent_alias_arn

Stored 'marketing_agent_id' (str)
Stored 'marketing_agent_alias_id' (str)
Stored 'marketing_agent_alias_arn' (str)
